In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

In [2]:
np.random.seed()
filename = '/notebooks/ML Project/BTC_USD_2013-10-01_2021-03-11-CoinDesk.csv'

In [3]:
Data = np.genfromtxt(filename, delimiter=',',skip_header=1) # not necessary
df = pd.read_csv(filename, sep = ',')
df.describe()

,Closing Price (USD),24h Open (USD),24h High (USD),24h Low (USD)
count,2708.000000,2708.000000,2708.000000,2708.000000
mean,5282.045337,5262.201010,5427.581541,5092.311449
std,7618.912012,7566.088241,7860.711886,7266.787750
min,108.584830,108.584830,118.675000,83.328330
25%,454.224500,454.015870,459.737415,445.667000
50%,2496.242490,2478.113745,2579.400000,2373.386250
75%,8169.154650,8166.425639,8348.647101,7915.519672
max,57128.642606,57453.358009,58332.364616,55531.550703


In [4]:

print(df)
print(type(df))
print(df.shape)

df = df.to_numpy() # not necessary
print(df)
print(type(df))
print(df.shape)

df = np.delete(df, [0,3,4,5], 1)
print(df)
print(type(df))
print(df.shape)

N = 7 # previous days as sepearete features
M = 2695 # number of data points



     Currency        Date  Closing Price (USD)  24h Open (USD)  \
0         BTC  2013-10-01           123.654990      124.304660   
1         BTC  2013-10-02           125.455000      123.654990   
2         BTC  2013-10-03           108.584830      125.455000   
3         BTC  2013-10-04           118.674660      108.584830   
4         BTC  2013-10-05           121.338660      118.674660   
...       ...         ...                  ...             ...   
2703      BTC  2021-03-07         48879.151904    48791.790509   
2704      BTC  2021-03-08         50594.698575    48887.566413   
2705      BTC  2021-03-09         51503.258132    50948.880707   
2706      BTC  2021-03-10         54458.037811    52360.748163   
2707      BTC  2021-03-11         56915.173935    54924.686532   

      24h High (USD)  24h Low (USD)  
0         124.751660     122.563490  
1         125.758500     123.633830  
2         125.665660      83.328330  
3         118.675000     107.058160  
4         121.936

In [5]:
# PREPROCESS
#######################################################
# Features and Label array
btcData = np.empty([M-N, N+1], dtype='float32')
# Labels array
# btcValues_y = np.empty([M-N, 1], dtype='float32')

# print(btcValues_x)
# print(btcValues_x.shape)
# print(btcValues_y)
# print(btcValues_y.shape)

# Create features array from df
for i in range(0, M-N):
    for j in range(i, i+N):
        btcData[i][j-i] = df[j][1]
    btcData[i][N] = df[i+N][1]
    
print(btcData)
#print(btcValues_y)

#######################################################



[[  123.65499   125.455     108.58483 ...   120.65533   121.795
    123.033  ]
 [  125.455     108.58483   118.67466 ...   121.795     123.033
    124.049  ]
 [  108.58483   118.67466   121.33866 ...   123.033     124.049
    125.96116]
 ...
 [48840.414   52165.3     51728.508   ... 57128.645   54181.914
  48172.88   ]
 [52165.3     51728.508   55719.203   ... 54181.914   48172.88
  48745.434  ]
 [51728.508   55719.203   54801.65    ... 48172.88    48745.434
  48291.41   ]]


In [6]:
#shuffle the data
np.random.seed()
np.random.shuffle(btcData)

In [7]:
print(btcData)
Y = btcData[:,N] #target vector
X = btcData[:,:N] #matrix of features

[[ 1189.366    1189.947    1193.611   ...  1285.376    1260.668
   1273.247  ]
 [  472.91      519.015     508.12875 ...   503.32126   504.62625
    513.39124]
 [47005.19    49151.168   48125.992   ... 51728.508   55719.203
  54801.65   ]
 ...
 [ 1036.099    1013.844     921.808   ...  1035.322    1011.969
   1041.374  ]
 [  613.675     623.6       620.59875 ...   607.03625   606.885
    606.8975 ]
 [  632.9533    630.3483    617.24335 ...   581.3333    570.3617
    562.9533 ]]


In [8]:
print(X)
print(Y)

[[ 1189.366    1189.947    1193.611   ...  1269.776    1285.376
   1260.668  ]
 [  472.91      519.015     508.12875 ...   501.63126   503.32126
    504.62625]
 [47005.19    49151.168   48125.992   ... 52165.3     51728.508
  55719.203  ]
 ...
 [ 1036.099    1013.844     921.808   ...   968.293    1035.322
   1011.969  ]
 [  613.675     623.6       620.59875 ...   604.8275    607.03625
    606.885  ]
 [  632.9533    630.3483    617.24335 ...   601.015     581.3333
    570.3617 ]]
[ 1273.247     513.39124 54801.65    ...  1041.374     606.8975
   562.9533 ]


In [9]:
# Split Data  Train + (Valid+Test)
X_train, X_tmp, y_train, y_tmp = train_test_split(X, Y, test_size = 0.4, random_state = 42)
X_test, X_valid, y_test, y_valid = train_test_split(X_tmp, y_tmp, test_size = 0.5, random_state = 42)

print(len(X_tmp), len(y_tmp))
print(len(X_test), len(y_test))
print(len(X_train), len(y_train))
print(len(X_valid), len(y_valid))

1076 1076
538 538
1612 1612
538 538


In [10]:
print(X_train)
print(y_train)

[[7363.2764  7490.621   7518.3335  ... 7716.145   7505.5894  7624.188  ]
 [3807.153   3587.1982  3865.5293  ... 3715.5645  3882.3047  3795.5928 ]
 [8237.343   8250.395   8493.131   ... 7951.5137  7614.964   7599.9526 ]
 ...
 [7436.835   7296.7964  7383.2173  ... 7491.6113  7363.2764  7490.621  ]
 [ 421.244    433.401    425.502   ...  436.426    434.001    419.254  ]
 [ 267.4691   274.25323  279.97906 ...  284.286    287.8309   300.53568]]
[6780.341  3812.381  7436.835  ... 7518.3335  421.597   304.3269]


In [11]:
# Linear Regresssion
#######################################################

model_linear = LinearRegression()
model_linear.fit(X_train, y_train)

LinearRegression()

In [12]:
# Polynomial Regresssion
#######################################################



In [13]:
# DEGREE = 1

model_poly1 = PolynomialFeatures(degree=1)
X_train_poly1 = model_poly1.fit_transform(X_train)

X_valid_poly1 = model_poly1.fit_transform(X_valid)

polyReg1 = LinearRegression()
polyReg1.fit(X_train_poly1, y_train)

prediction_training_poly1 = polyReg1.predict(X_train_poly1)

prediction_valid_poly1 = polyReg1.predict(X_valid_poly1)

In [14]:
# DEGREE = 2

model_poly2 = PolynomialFeatures(degree=2)
X_train_poly2 = model_poly2.fit_transform(X_train)

X_valid_poly2 = model_poly2.fit_transform(X_valid)

polyReg2 = LinearRegression()
polyReg2.fit(X_train_poly2, y_train)

prediction_training_poly2 = polyReg2.predict(X_train_poly2)

prediction_valid_poly2 = polyReg2.predict(X_valid_poly2)

In [15]:
# DEGREE = 3

model_poly3 = PolynomialFeatures(degree=3)
X_train_poly3 = model_poly3.fit_transform(X_train)

X_valid_poly3 = model_poly3.fit_transform(X_valid)

polyReg3 = LinearRegression()
polyReg3.fit(X_train_poly3, y_train)

prediction_training_poly3 = polyReg3.predict(X_train_poly3)

prediction_valid_poly3 = polyReg3.predict(X_valid_poly3)

In [16]:
btc_y_predict = model_linear.predict(X_valid)
btc_y_predict_training = model_linear.predict(X_train)


In [17]:
# The coefficients
print(X_train.shape)
print(X_valid.shape)


print(y_valid[30:40])
print(btc_y_predict[30:40])

(1612, 7)
(538, 7)
[  560.195   7790.3213 11744.013   9148.445    597.585  48172.88
 10829.505   9757.297   7303.406    985.9387]
[  524.24445  7651.3813  11623.418    9143.084     591.78937 55893.293
 10750.709    9321.279    7281.0366   1125.2765 ]


In [18]:
print('Coefficients: \n')

print('Linear')
model_linear.coef_

Coefficients: 

Linear


array([-0.26601824,  0.20952088,  0.05074604, -0.02445187, -0.06705038,
        0.1974884 ,  0.90623367], dtype=float32)

In [19]:
model_linear.intercept_

-15.812012

In [20]:
print('Poly1')
polyReg1.coef_

Poly1


array([ 0.        , -0.26601866,  0.20952162,  0.05074392, -0.0244497 ,
       -0.06704929,  0.19748959,  0.9062314 ], dtype=float32)

In [21]:
print('Poly2')
polyReg2.coef_

Poly2


array([-8.3396560e-01,  1.9963312e-01,  2.6014661e-02, -2.3822007e-01,
       -2.5421274e-01,  2.9094260e-02,  4.7240320e-01,  7.7308106e-01,
        2.1734461e-04,  2.7640909e-04, -6.1002374e-04,  4.1959435e-04,
        4.4073537e-04, -8.0316514e-04, -1.5176833e-04, -3.8204715e-04,
        2.7491897e-04, -1.5461780e-03,  1.0720380e-03, -8.9259073e-04,
        1.5438907e-03,  3.5227835e-04,  1.9683167e-03, -1.1755861e-03,
       -2.2783875e-05, -1.0695048e-03, -9.3062222e-04,  3.2791123e-04,
        9.9946931e-04, -3.4337491e-04, -2.5762618e-04, -1.2968481e-04,
        4.9587339e-05, -2.3453310e-04,  1.2075007e-03, -6.0022622e-04],
      dtype=float32)

In [22]:
print('Poly3')
polyReg3.coef_

Poly3


array([ 1.51753682e-06, -3.36854873e-06, -2.49125520e-08,  5.69689469e-08,
        1.73914554e-08,  1.88222771e-08,  7.48000950e-09,  1.27808875e-10,
        2.43698582e-06,  2.58235696e-06,  2.50270887e-06,  2.49427421e-06,
        2.35204880e-06,  2.53945291e-06,  2.60679417e-06,  2.75613615e-06,
        2.68268445e-06,  2.67026280e-06,  2.51801771e-06,  2.70422379e-06,
        2.76897390e-06,  2.62108119e-06,  2.61007199e-06,  2.46025525e-06,
        2.64992309e-06,  2.71676845e-06,  2.61927789e-06,  2.47153707e-06,
        2.66549091e-06,  2.72793341e-06,  2.33036189e-06,  2.52340783e-06,
        2.58572845e-06,  2.72887337e-06,  2.79987580e-06,  2.87632020e-06,
        6.70950158e-08,  2.27640658e-06, -3.02273133e-06, -6.97241092e-07,
        1.17641707e-06, -1.87121805e-06,  1.98096677e-06, -2.74080708e-06,
        3.57516114e-06, -1.45664330e-07, -1.30219951e-06,  2.02704246e-06,
       -3.25511496e-06,  1.37720235e-07,  2.14820034e-07,  8.36460629e-07,
        8.52974608e-07,  

In [23]:
print('Mean squared error <Training>: ')

print('Linear')
mean_squared_error(y_train, btc_y_predict_training)

Mean squared error <Training>: 
Linear


166259.27

In [24]:
print('Poly1')
mean_squared_error(y_train, prediction_training_poly1)

Poly1


166259.23

In [25]:
print('Poly2')
mean_squared_error(y_train, prediction_training_poly2)

Poly2


2930132.8

In [26]:
print('Poly3')
mean_squared_error(y_train, prediction_training_poly3)

Poly3


12012907.0

In [27]:
print('Mean squared error <Valid>: ')

print('Linear')
mean_squared_error(y_valid, btc_y_predict)

Mean squared error <Valid>: 
Linear


316005.7

In [28]:
print('Poly1')
mean_squared_error(y_valid, prediction_valid_poly1)

Poly1


316007.38

In [29]:
print('Poly2')
mean_squared_error(y_valid, prediction_valid_poly2)

Poly2


1393093.8

In [30]:
print('Poly3')
mean_squared_error(y_valid, prediction_valid_poly3)

Poly3


294644260.0

In [31]:
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination <Training>: ')

print('Linear')
r2_score(y_train, btc_y_predict_training)

Coefficient of determination <Training>: 
Linear


0.9969374372065298

In [32]:
print('Poly1')
r2_score(y_train, prediction_training_poly1)

Poly1


0.9969374375713302

In [33]:
print('Poly2')
r2_score(y_train, prediction_training_poly2)

Poly2


0.9460257684396932

In [34]:
print('Poly3')
r2_score(y_train, prediction_training_poly3)

Poly3


0.7787173809834796

In [35]:
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination <Valid>: ')

print('Linear')
r2_score(y_valid, btc_y_predict)

Coefficient of determination <Valid>: 
Linear


0.9932259446833832

In [36]:
print('Poly1')
r2_score(y_valid, prediction_valid_poly1)

Poly1


0.9932259089438289

In [37]:
print('Poly2')
r2_score(y_valid, prediction_valid_poly2)

Poly2


0.9701369517133717

In [38]:
print('Poly3')
r2_score(y_valid, prediction_valid_poly3)

Poly3


-5.316141138586597

In [39]:
# Experiment

In [40]:
btc_y_predict_training

array([7664.004  , 3786.2263 , 7607.037  , ..., 7475.395  ,  410.20145,
        287.8288 ], dtype=float32)

In [41]:
y_train

array([6780.341 , 3812.381 , 7436.835 , ..., 7518.3335,  421.597 ,
        304.3269], dtype=float32)

In [42]:
# Chosen model Linear Regression: according to lowest validation error

In [43]:
# Test final performance of the chosen model on separate Test Set

In [44]:
# MSE

In [45]:
btc_y_predict_test = model_linear.predict(X_test)

In [46]:
print('Mean squared error <Test>: ')

print('Linear')
mean_squared_error(y_test, btc_y_predict_test)

Mean squared error <Test>: 
Linear


109240.445

In [47]:
# R2 Score

In [48]:
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination <Test>: ')

print('Linear')
r2_score(y_test, btc_y_predict_test)

Coefficient of determination <Test>: 
Linear


0.996813471173821